<h1><center>Model UC defect points equations</center></h1>

<h2>Imports and helper functions</h2>

In [6]:
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [16]:
#Contans
pi = np.pi
kb = 8.617333262145e-5  # Boltzmann constant in eV/K
T = 600 + 273.15        # Temperature in Kelvin
filename = "../../data/UC-UN/UC-diameter-density.csv"

In [8]:
# Helper functions
def D_Coeff(D_0, E_m, T):
    return D_0 * np.exp(-E_m / (kb * T))

def log_factor(r_0, r_L, eps=1e-10):
    logt = np.log(8 * r_L / r_0)
    r = r_0 * logt
    return 1 / r 

def compute_j_L_v(C_Ce_v, C_O_v, D_Ce_v, D_O_v, r_0, r_L, eps=1e-10):
    log_fact = log_factor(r_0, r_L)
    num = D_Ce_v * C_Ce_v * D_O_v * C_O_v
    den = 2 * D_Ce_v * C_Ce_v + D_O_v * C_O_v
    return log_fact * num / den 

def compute_j_L_i(C_Ce_i, C_O_i, D_Ce_i, D_O_i, r_0, r_L, eps=1e-10):
    log_fact = log_factor(r_0, r_L)
    num = D_Ce_i * C_Ce_i * D_O_i * C_O_i
    den = 2 * D_Ce_i * C_Ce_i + D_O_i * C_O_i
    return log_fact * num / den 

def compute_j_ii(C_Ce_i, C_O_i, D_Ce_i, D_O_i, eps=1e-10):
    num = D_Ce_i * (C_Ce_i**2) * D_O_i * (C_O_i**2)
    den = 2 * D_Ce_i * (C_Ce_i**2) + D_O_i * (C_O_i**2)
    return num / den 

In [9]:
def load_uc_datasets(filename: str):
    # Read CSV file, skipping first row (dataset names row)
    df = pd.read_csv(filename, skiprows=1)

    # Temperature mapping
    temp_map_uc = {
        '300': '300°C',
        '600': '600°C'
    }

    datasets_uc = {}

    # Column mapping
    temperatures = ['300', '600']
    density_cols = [0, 2]   # X columns for density data
    diameter_cols = [4, 6]  # X columns for diameter data

    # Helper function to extract one dataset into DataFrame
    def extract_dataframe(x_col, y_col, label):
        x_data = pd.to_numeric(df.iloc[:, x_col], errors='coerce').dropna()
        y_data = pd.to_numeric(df.iloc[:, y_col], errors='coerce').dropna()
        min_len = min(len(x_data), len(y_data))
        if min_len == 0:
            return None
        
        x_vals = x_data.iloc[:min_len].values
        y_vals = y_data.iloc[:min_len].values
        
        sorted_idx = np.argsort(x_vals)
        x_sorted = x_vals[sorted_idx]
        y_sorted = y_vals[sorted_idx]
        
        return pd.DataFrame({
            "X": x_sorted,
            "Y": y_sorted,
            "temp": temp_map_uc[label]
        })

    # Extract density datasets
    for i, temp in enumerate(temperatures):
        result = extract_dataframe(density_cols[i], density_cols[i] + 1, temp)
        if result is not None:
            datasets_uc[f"density_uc_{temp}"] = result

    # Extract diameter datasets
    for i, temp in enumerate(temperatures):
        result = extract_dataframe(diameter_cols[i], diameter_cols[i] + 1, temp)
        if result is not None:
            datasets_uc[f"diameter_uc_{temp}"] = result

    return datasets_uc


<h2>ODE System</h2>

In [ ]:
# ODE system
def ODE_system(t, y, params):
    C_Th_v, C_O_v, C_Th_i, C_O_i, N_L, R_L = y

    G_Th_v = params['G_Th_v']
    G_O_v  = params['G_O_v']
    G_Th_i = params['G_Th_i']
    G_O_i  = params['G_O_i']
    a     = params['a']
    Omega_0 = (a**3) / 12
    D_Th_i = params['D_Th_i']
    D_O_i  = params['D_O_i']
    D_Th_v = params['D_Th_v']
    D_O_v  = params['D_O_v']
    r0     = params['r0']
    L = params['L']
    b = a / np.sqrt(3)
    alpha = params['alpha']
    beta = params['beta']
    k_lf = params['k_lf']
    eta =  Omega_0 / a**2
    st   = 2 * pi * R_L * N_L

     
    j_L_i = 0
    j_ii = 0
    
    if C_Th_i != 0 or C_O_i != 0:
        j_L_i = compute_j_L_i(C_Th_i, D_Th_i, r0, R_L)
        j_ii  = compute_j_ii(C_Th_i, C_O_i, D_Th_i, D_O_i, eta)
    
    tau = loop_life_time_term(N_L, R_L, alpha, beta, k_lf)
    
    k_Th = eta * (48 * D_Th_i + 48 * D_Th_i)
    k_O  = eta * (36 * D_O_i + 24 * D_O_i)

    c_lamalle = 12/(L**2)

    
    dc = np.zeros(6)
    
    #R_L evolution
    
    dc[5] = 3 * Omega_0 * 2 * pi * r0 * j_L_i / b

    dc[4] =  j_ii 

    if tau != 0 :
        dc[4] -=  N_L / tau
    

    
    if dc[5] < 0 and R_L <= 2*r0:
        dc[5] = 0

    if dc[4] < 0 and N_L <= 0:
        dc[4] = 0

    dc[0] = G_Th_v - k_Th * C_Th_v * C_Th_i # - c_lamalle * D_Th_v * C_Th_v (neglected)
    dc[1] = G_O_v  - k_O  * C_O_v  * C_O_i  - c_lamalle * D_O_v * C_O_v
    dc[2] = G_Th_i - k_Th * C_Th_i * C_Th_v - c_lamalle * D_Th_i * C_Th_i  - j_L_i * pi * r0 * st - j_ii
    dc[3] = G_O_i  - k_O  * C_O_i  * C_O_v  - c_lamalle * D_O_i * C_O_i - 2 * j_L_i * pi * r0 * st - 2 * j_ii

    return dc

<h2>Fit function</h2>

In [ ]:
def fit_function(E_Th_i, E_O_v, E_O_i, k_lf,
                 dose_exp_R, dose_exp_N, y0, t_span, ode_params, G_0, t_eval):

    E_Th_v = 5.0  # fixed

    ode_params['D_Th_i'] = D_Coeff(D_0_Th_i, E_Th_i, T)
    ode_params['D_Th_v'] = D_Coeff(D_0_Th_v, E_Th_v, T)
    ode_params['D_O_i']  = D_Coeff(D_0_O_i,  E_O_i, T)
    ode_params['D_O_v']  = D_Coeff(D_0_O_v,  E_O_v, T)
    ode_params['k_lf']   = k_lf

    # Solve ODE
    sol = solve_ivp(
        ODE_system, t_span, y0, args=(ode_params,),
        method="BDF", t_eval=t_eval, rtol=1e-6, atol=1e-9
    )

    # Check solver success
    if not sol.success:
        print("Solver failed:", sol.message)
        # Return NaNs so curve_fit knows this set of parameters is bad
        return np.full(len(dose_exp_R) + len(dose_exp_N), np.nan)

    dose_model = sol.t * G_0

    R_model = np.interp(dose_exp_R, dose_model, sol.y[5, :])  # radius
    N_model = np.interp(dose_exp_N, dose_model, sol.y[4, :])  # density

    return np.concatenate([R_model, N_model])

<h2>Doing the fit</h2>
<h3>Defining parameters</h3>

In [ ]:
#Parameters
a = 541e-10   # cm
N = 12 
Omega_0 = (a**3) / N
b = a / np.sqrt(3)

D_0_Ce_v, D_0_Ce_i, D_0_O_v, D_0_O_i = 0.65, 0.01, 0.02, 0.01
E_m_Ce_v, E_m_Ce_i, E_m_O_v, E_m_O_i = 5.3, 2.6, 0.836, 1.18

# Dose rate dpa/s
G_0 = 0.87e-6 #LF

ratio = 2
G_tot = G_0 / Omega_0
G_Ce = G_tot * (ratio / (1 + ratio))
G_O  = G_Ce   * (1 - ratio / (1 + ratio))
# Initial conditions
t0 = 0.0001149
y0 = [G_Th*t0, G_O*t0, G_Th*t0, G_O*t0, G_O*t0*1e-6, np.sqrt(N*Omega_0/(pi*b))]

#Damage rate dpa/s
G_0 = 1.9e-3

# Total time from total dose
dmg = 3.0   # dpa
t_end = dmg / G_0
t_span = (t0, t_end)
t_eval = np.linspace(t0, t_end, 3000)
params = {
    'G_Ce_v': G_Ce, 'G_O_v': G_O,
    'G_Ce_i': G_Ce, 'G_O_i': G_O,
    'a': a, 
    'D_Ce_i': D_Coeff(D_0_Ce_i, E_m_Ce_i, T),
    'D_Ce_v': D_Coeff(D_0_Ce_v, E_m_Ce_v, T),
    'D_O_i':  D_Coeff(D_0_O_i,  E_m_O_i, T),
    'D_O_v':  D_Coeff(D_0_O_v,  E_m_O_v, T),
    'r0': 4*a,
}

In [ ]:

datasets_uc = load_uc_datasets(filename)


In [ ]:
# Initial conditions
t0 = 0.0001149
y0 = [G_Th*t0, G_O*t0, G_Th*t0, G_O*t0, G_O*t0*1e-6, np.sqrt(N*Omega_0/(pi*b))]

#Damage rate dpa/s
G_0 = 1.9e-3
# Combine experimental data into one array
data_all = np.concatenate([R_exp, N_exp])

# Initial guess and bounds
p0 = [0.836, 1.8, 0.6, 5.0]  #E_Th_i, E_O_v, E_O_i, k_lf,
bounds = ([0.5, 0.5, 0.5, 0], [4.0, 4.0, 4.0, 8])

# Perform fit
popt, pcov = curve_fit(
    lambda _, E_Th_i, E_O_v, E_O_i, k_lf: 
        fit_function(E_Th_i, E_O_v, E_O_i, k_lf,
                     dose_exp_R, dose_exp_N, y0, t_span, ode_params.copy(), G_0, t_eval),
    xdata=np.zeros_like(data_all), 
    ydata=data_all, 
    p0=p0, bounds=bounds
)

print("Best fit parameters:")
print(f"E_Th_i = {popt[0]:.3f} eV")
print(f"E_O_v  = {popt[1]:.3f} eV")
print(f"E_O_i  = {popt[2]:.3f} eV")
print(f"k_lf   = {popt[3]:.3e}")
# Total time from total dose
dmg = 3.0   # dpa
t_end = dmg / G_0
t_span = (t0, t_end)
t_eval = np.linspace(t0, t_end, 3000)